The dataset for running the inference can be retrieved from [Udacity Project Repo](https://github.com/udacity/AIHCND_C2_Starter). 

In [2]:
import keras
from keras.models import model_from_json
from skimage.transform import resize
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline


Using TensorFlow backend.


In [3]:
def check_dicom(filename):
    """
    This function reads in a .dcm file, checks the important fields for our 
    device, and returns a numpy array of just the imaging data

    Args:
        filename (str): .dcm file name

    Returns:
        img (numpy array): numpy array of the imaging data
        ds.StudyDescription : StudyDescription filed from the meta-data. 
    """
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)

    if ds.PatientPosition not in ['PA', 'AP']:
        print("File {} contains invalid image position {}.".format(
            filename, ds.PatientPosition))
        return None, None
    if ds.Modality != 'DX':
        print("File {} contains invalid image type {}.".format(
            filename, ds.Modality))
        return None, None
    if ds.BodyPartExamined != 'CHEST':
        print("File {} contains invalid body part {}.".format(
            filename, ds.BodyPartExamined))
        return None, None

    img = ds.pixel_array
    print('File {} was successfully loaded.'.format(filename))
    return img, ds.StudyDescription


In [4]:
#
def preprocess_image(img, img_size=(1, 224, 224, 3)):
    """
    This function takes the numpy array output by check_dicom and runs the 
        appropriate pre-processing needed for our model input

    Args:
        img (numpy array): numpy array of the converted DICOM image
        img_size (tuple, optional): dimensions used to resize the image. 
            Defaults to (1, 224, 224, 3).

    Returns:
        proc_img (numpy array): numpy array of the resized image.
    """
    resize_img = resize(img, img_size)
    # normalization
    img_std = 255
    proc_img = (resize_img - resize_img.mean())/img_std

    return proc_img

# This function loads in our trained model w/ weights and compiles it
def load_model(experiment):
    with open(f"exp{experiment}_model.json", "r") as json_file:
        loaded_model_json = json_file.read()
    model = model_from_json(loaded_model_json)
    model.load_weights(f"xray_class_model{experiment}.best.hdf5")
    print("Loaded model from disk.")
    return model

# Inference
def predict_image(model, img, thresh):
    if model.predict(img) > thresh:
        msg = "Pneumonia positive."
    else:
        msg = "Pneumonia negative."
    return msg


In [5]:
test_dicoms = ['test1.dcm', 'test2.dcm', 'test3.dcm',
               'test4.dcm', 'test5.dcm', 'test6.dcm']

my_model = load_model(2)
my_model.summary()
thresh = 0.302

# use the .dcm files to test the model's prediction
for i in test_dicoms:

    img = np.array([])
    img, label = check_dicom(i)

    if img is None:
        continue

    img_proc = preprocess_image(img)
    pred = predict_image(my_model, img_proc, thresh)
    print('Model Prediction: {}\nLabel: {}\n'.format(pred, label))


Loaded model from disk.
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None,